## 第 0000 题

### 描述

将你的 QQ 头像（或者微博头像）右上角加上红色的数字，类似于微信未读信息数量那种提示效果。 类似于图中效果
![](https://camo.githubusercontent.com/d518d3929e4054ce2f9183b23e52908da7e5632d/687474703a2f2f692e696d6775722e636f6d2f736732646b75592e706e673f31)

### 分步

In [ ]:
%matplotlib inline

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from matplotlib.pyplot import imshow
import numpy as np

In [ ]:
def show_pic(im):
    imshow(np.asarray(im))

In [ ]:
im = Image.open("pictures/girl.jpg").convert('RGBA')

In [ ]:
show_pic(im)

1、调整图像大小

In [ ]:
im = im.resize((200, 200))

In [ ]:
im

2、在右上角画图

In [ ]:
# 创建绘图对象
draw = ImageDraw.Draw(im)

In [ ]:
# 为文本创建绘图对象
txt = Image.new('RGBA', im.size, (255, 255, 255, 0))

In [ ]:
# 设置字体
fnt = ImageFont.truetype(font="./FreeMono.ttf", size=40)

In [ ]:
# 为文本图层创建绘图上下文
d = ImageDraw.Draw(txt)

In [ ]:
# 绘制文字
d.text((160, 20), "4", font=fnt, fill=(255, 0, 0, 255))

In [ ]:
# 合并图像
out = Image.alpha_composite(im, txt)

In [ ]:
out

3、保存图片

In [ ]:
out = out.convert('RGB')

In [ ]:
out.save('pictures/girl_with_message.jpg')

4、关闭文件

In [ ]:
im.close()
out.close()

### 操场

In [ ]:
help(ImageDraw.Draw)

In [ ]:
help(im.resize)

In [ ]:
help(draw.textsize)

In [ ]:
help(draw.text)

In [ ]:
help(ImageFont.truetype)

### 小结

1. Ubuntu 字体文件位于 /usr/share/fonts
2. PIL 简单用法

In [ ]:
from PIL import Image, ImageDraw, ImageFont

im = Image.open("pictures/girl.jpg")
im = im.resize((200, 200))
w, h = im.size

fnt = ImageFont.truetype(font="fonts/FreeMono.ttf", size=40)
draw = ImageDraw.Draw(im)
draw.text((5/6 * w, 1/15 * h), '5', fill=(255, 10, 10), font=fnt)

im.save('pictures/girl_with_message.jpg')
im.close()

## 第 0001 题

### 描述

做为 Apple Store App 独立开发者，你要搞限时促销，为你的应用生成激活码（或者优惠券），使用 Python 如何生成 200 个激活码（或者优惠券）？

### 分步

In [ ]:
import string
import random
from pprint import pprint

In [ ]:
valid_chars = string.ascii_letters + string.digits


def gen_coupons(count=200, length=6, unique=True):
    ret = set()
    while len(ret) < count:
        if unique:
            ret.add(''.join(random.sample(valid_chars, length)))
        else:
            ret.add(''.join(valid_chars[random.randint(0, len(valid_chars) - 1)] for i in range(length)))
    return list(ret)

In [ ]:
coupons = gen_coupons(10, 6, False)

In [ ]:
pprint(coupons)

In [ ]:
with open('data/coupons.txt', 'w') as f:
    f.write('\n'.join(gen_coupons()))

### 操场

In [ ]:
li = [x for x in range(10)]

从序列中挑选出对应数目的样本，不会重复

In [ ]:
random.sample(li, 3)

重新排列序列的顺序

In [ ]:
random.shuffle(li)
print(li)

从序列中选出一个元素

In [ ]:
random.choice(li)

### 小结

In [ ]:
import string
import random

valid_chars = string.ascii_letters + string.digits

def gen_coupons(count=200, length=6, unique=True):
    ret = set()
    while len(ret) < count:
        if unique:
            ret.add(''.join(random.sample(valid_chars, length)))
        else:
            ret.add(''.join([random.choice(valid_chars) for i in range(length)]))
    
    return list(ret)


coupons = gen_coupons(10, 6)
pprint(coupons)

## 第 0002 题

### 描述

将 0001 题生成的 200 个激活码（或者优惠券）保存到 MySQL 关系型数据库中。

### 分步

In [1]:
import configparser
import mysql.connector as db

1、读取 mysql 配置文件，为了安全性不应该将配置信息硬编码在程序中。

In [2]:
cf = configparser.ConfigParser()
cf.read("/home/lfish/.config/mysql.cnf")

if not cf.has_section("mysql"):
    raise Exception("Section not found.")
    
try:
    localhost=cf.get('mysql', 'host')
    username=cf.get('mysql', 'user')
    password=cf.get('mysql', 'passwd')
except Exception as err:
    raise err

2、连接数据库。

In [32]:
mydb = db.connect(host=localhost, user=username, passwd=password, database="show_me_the_code")

3、查看数据库。

In [33]:
mycursor = mydb.cursor()

In [24]:
mycursor.execute("show databases")

In [25]:
for database in mycursor:
    print(database)

('information_schema',)
('detail',)
('mysql',)
('performance_schema',)
('show_me_the_code',)
('simple',)
('sys',)


3、创建数据库。

In [ ]:
mycursor.execute("create database show_me_the_code")

4、使用数据库。

In [7]:
mycursor.execute("use show_me_the_code")

5、删除数据库。

In [10]:
mycursor.execute("drop table coupons")

6、创建表。

In [11]:
mycursor.execute("create table coupons (id INT AUTO_INCREMENT PRIMARY KEY, code varchar(6))")

7、插入数据到数据库。

In [12]:
sql = "insert into coupons (code) values (%s)"

In [13]:
with open("data/coupons.txt") as f:
    coupons = [(x.strip(), ) for x in f if x]

In [14]:
mycursor.executemany(sql, coupons)

8、查看插入的数据。

In [31]:
mycursor.execute("select * from coupons")

InternalError: Unread result found

In [27]:
for coupon in coupons:
    if '6' in coupon[0]:
        print(coupon[0])

pI6FSL
6VhMmT
57WJ6b
VbdML6
E1k6Yz
e4h6l0
8uRZ6N
ZkfjI6
ifE6gw
wlm6RE
OIU6Qg
lV6fhR
algfb6
67zpwO
asgVm6
w6rkWK


### 操场

In [34]:
sql = "INSERT INTO coupons (code) VALUES (%s)"
val = "bbbb",

In [40]:
mycursor.execute("INSERT INTO coupons (code) VALUES (%s)", val)

InternalError: Unread result found